In [1]:
from netCDF4 import Dataset
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
class ParticleData(object):
    def __init__(self):
        self.id = []

def load_particles_file(fname, varnames):
    T = ParticleData()
    pfile = Dataset(fname, 'r')
    T.id = pfile.variables['trajectory'][:]
    for v in varnames:
        setattr(T, v, pfile.variables[v][:])
    T.time /= 86400.  # convert to days
    T.time_origin = datetime.datetime(2001, 1, 1)
    return T

T = load_particles_file("corefootprint_particles.nc", ['lon', 'lat', 'temp', 'z', 'time'])

In [4]:
pylab.rcParams['figure.figsize'] = (10, 6)
def makemov(T):
    fig = plt.figure()

    frames = np.unique(T.time)
    mintemp = np.amin([T.temp[i] for i in range(len(T.time)) if T.z[i] == 50.])
    maxtemp = np.amax([T.temp[i] for i in range(len(T.time)) if T.z[i] == 50.])

    def settitle(time):
        titstr = 'Particles on ' + (T.time_origin + datetime.timedelta(days=time)).strftime("%Y-%m-%d")
        return titstr

    ax = fig.add_subplot(1, 1, 1)
    ttl = ax.set_title(settitle(int(frames[0])))
    m = Basemap(projection='merc', llcrnrlat=-40, urcrnrlat=-27.5, llcrnrlon=10, urcrnrlon=32.5, resolution='h')
    m.drawcoastlines()
    m.fillcontinents(color='burlywood')
    m.drawparallels(np.arange(-50, -20, 10), labels=[True, False, False, False])
    m.drawmeridians(np.arange(0, 40, 10), labels=[False, False, False, True])

    I = [i for i in range(len(T.time)) if T.time[i]==frames[0] and T.z[i] > 50.]
    xs, ys = m(T.lon[I], T.lat[I])
    scat_sink = m.scatter(xs, ys, c='k', s=15)

    I = [i for i in range(len(T.time)) if T.time[i]==frames[0] and T.z[i] == 50.]
    xs, ys = m(T.lon[I], T.lat[I])
    scat_dwell = m.scatter(xs, ys, c=T.temp[I], s=35, edgecolor='k')
    scat_dwell.set_clim(mintemp, maxtemp)
    plt.colorbar(scat_dwell)

    def animate(t):
        I = [i for i in range(len(T.time)) if T.time[i]==t and T.z[i] > 50.]
        scat_sink.set_offsets(np.matrix(m(T.lon[I], T.lat[I])).transpose())

        I = [i for i in range(len(T.time)) if T.time[i]==t and T.z[i] == 50.]
        scat_dwell.set_offsets(np.matrix(m(T.lon[I], T.lat[I])).transpose())
        scat_dwell.set_array(T.temp[I])
        ttl.set_text(settitle(int(t)))
        return scat_sink

    rc('animation', html='html5')
    anim = animation.FuncAnimation(fig, animate, frames=frames,
                                   interval=100, blit=False)
    plt.close()
    return anim

makemov(T)